# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [3]:
import graphlab

In [64]:
import numpy as np

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [62]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature.apply(lambda x: x**power)
    return poly_sframe
    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to kushalkanungo1991@gmail.com and will expire on July 04, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\kaykay\AppData\Local\Temp\graphlab_server_1486711665.log.0


As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [6]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [ ]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [13]:
abc = []
c = 0
s = []
mod=[]
for x in range(1,16):
    poly = polynomial_sframe(sales['sqft_living'], x)
    #val = polynomial_sframe(validation['sqft_living'], x)
    my_features = poly.column_names() # get the name of the features
    poly['price'] = sales['price'] # add price to the data since it's the target in training set
    #val['price'] = validation['price'] # add price to the data since it's the target in validation set
    model = graphlab.linear_regression.create(poly, target = 'price', l2_penalty=1e-5,features = my_features, validation_set = None, verbose = False)
    mod.append(model)
    """z = model.evaluate(val)
    print z['rmse']
    s.append([z['rmse'],x])
    if (c == 0 ):
        abc = [z['rmse'],x,model]
        c = 1
    else:
        if z['rmse']<abc[0]:
            abc = [z['rmse'],x,model]"""

In [14]:
mod


[Class                          : LinearRegression
 
 Schema
 ------
 Number of coefficients         : 2
 Number of examples             : 21613
 Number of feature columns      : 1
 Number of unpacked features    : 1
 
 Hyperparameters
 ---------------
 L1 penalty                     : 0.0
 L2 penalty                     : 0.0
 
 Training Summary
 ----------------
 Solver                         : newton
 Solver iterations              : 1
 Solver status                  : SUCCESS: Optimal solution found.
 Training time (sec)            : 0.014
 
 Settings
 --------
 Residual sum of squares        : 1.47727636709e+15
 Training RMSE                  : 261440.7907
 
 Highest Positive Coefficients
 -----------------------------
 power_1                        : 280.6236
 
 Lowest Negative Coefficients
 ----------------------------
 (intercept)                    : -43580.7387,
 Class                          : LinearRegression
 
 Schema
 ------
 Number of coefficients         : 3
 Number 

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [15]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [20]:
abc = []
c = 0
s = []
mod=[]
x = 15
poly_1 = polynomial_sframe(set_1['sqft_living'], x)
poly_2 = polynomial_sframe(set_2['sqft_living'], x)
poly_3 = polynomial_sframe(set_3['sqft_living'], x)
poly_4 = polynomial_sframe(set_4['sqft_living'], x)
#val = polynomial_sframe(validation['sqft_living'], x)
my_features = poly_1.column_names() # get the name of the features
poly_1['price'] = set_1['price'] # add price to the data since it's the target in training set
poly_2['price'] = set_2['price']
poly_3['price'] = set_3['price']
poly_4['price'] = set_4['price']
#val['price'] = validation['price'] # add price to the data since it's the target in validation set
model_1 = graphlab.linear_regression.create(poly_1, target = 'price', l2_penalty=1e-5,features = my_features, validation_set = None, verbose = False)
model_2 = graphlab.linear_regression.create(poly_2, target = 'price', l2_penalty=1e-5,features = my_features, validation_set = None, verbose = False)
model_3 = graphlab.linear_regression.create(poly_3, target = 'price', l2_penalty=1e-5,features = my_features, validation_set = None, verbose = False)
model_4 = graphlab.linear_regression.create(poly_4, target = 'price', l2_penalty=1e-5,features = my_features, validation_set = None, verbose = False)

mod = [model_1,model_2,model_3,model_4]
    

In [30]:
mod[0]['coefficients']

name,index,value,stderr
(intercept),None,9306.4666294,810199.413847
power_1,None,585.865804883,3442.10422346
power_2,None,-0.397305874315,6.02291333786
power_3,None,0.000141470888681,0.0057279761005
power_4,None,-1.52945955404e-08,3.29578977423e-06
power_5,None,-3.79756819869e-13,1.2133445418e-09
power_6,None,5.97481991752e-17,3.00417024061e-13
power_7,None,1.06888523817e-20,5.50094917614e-17
power_8,None,1.59343780943e-25,8.36424155782e-21
power_9,None,-6.92834881362e-29,1.00031911962e-24


In [31]:
mod[1]['coefficients']

name,index,value,stderr
(intercept),None,-25115.8993262,1461908.2824
power_1,None,783.493780749,8141.54862382
power_2,None,-0.767759273211,19.1599476904
power_3,None,0.000438766345759,0.0252348495278
power_4,None,-1.15169156529e-07,2.07713736269e-05
power_5,None,6.84281125926e-12,1.1267705289e-08
power_6,None,2.51195199365e-15,4.11595289605e-12
power_7,None,-2.0644054241e-19,1.00652509326e-15
power_8,None,-4.59673169746e-23,1.56967454249e-19
power_9,None,-2.71272039208e-29,1.26503325439e-23


In [32]:
mod[2]['coefficients']

name,index,value,stderr
(intercept),None,462426.567754,1264112.98769
power_1,None,-759.251854206,6160.31746987
power_2,None,1.02867006808,12.6575879778
power_3,None,-0.00052826454477,0.0145109950169
power_4,None,1.15422915761e-07,1.03360324025e-05
power_5,None,-2.26096108253e-12,4.79760227635e-09
power_6,None,-2.08214269714e-15,1.46673002389e-12
power_7,None,4.08770281089e-20,2.86235422373e-16
power_8,None,2.57079169816e-23,3.16708303132e-20
power_9,None,1.2431125129e-27,2.07861266245e-24


In [33]:
mod[3]['coefficients']

name,index,value,stderr
(intercept),None,-170240.034765,1038851.01649
power_1,None,1247.59034807,5068.70200366
power_2,None,-1.22460911823,5.25072710262
power_3,None,0.000555254617758,nan
power_4,None,-6.38262314137e-08,nan
power_5,None,-2.20216008557e-11,nan
power_6,None,4.8183470755e-15,nan
power_7,None,4.21461639711e-19,nan
power_8,None,-7.99880731487e-23,nan
power_9,None,-1.32365918453e-26,nan


The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [34]:
x = 15
poly_1 = polynomial_sframe(set_1['sqft_living'], x)
poly_2 = polynomial_sframe(set_2['sqft_living'], x)
poly_3 = polynomial_sframe(set_3['sqft_living'], x)
poly_4 = polynomial_sframe(set_4['sqft_living'], x)
#val = polynomial_sframe(validation['sqft_living'], x)
my_features = poly_1.column_names() # get the name of the features
poly_1['price'] = set_1['price'] # add price to the data since it's the target in training set
poly_2['price'] = set_2['price']
poly_3['price'] = set_3['price']
poly_4['price'] = set_4['price']
#val['price'] = validation['price'] # add price to the data since it's the target in validation set
model_1 = graphlab.linear_regression.create(poly_1, target = 'price', l2_penalty=1e5,features = my_features, validation_set = None, verbose = False)
model_2 = graphlab.linear_regression.create(poly_2, target = 'price', l2_penalty=1e5,features = my_features, validation_set = None, verbose = False)
model_3 = graphlab.linear_regression.create(poly_3, target = 'price', l2_penalty=1e5,features = my_features, validation_set = None, verbose = False)
model_4 = graphlab.linear_regression.create(poly_4, target = 'price', l2_penalty=1e5,features = my_features, validation_set = None, verbose = False)

mod = [model_1,model_2,model_3,model_4]

In [37]:
mod[0]['coefficients']

name,index,value,stderr
(intercept),None,530317.024516,1219250.91739
power_1,None,2.58738875673,5179.94540664
power_2,None,0.00127414400592,9.06374713073
power_3,None,1.74934226932e-07,0.00861990270049
power_4,None,1.06022119097e-11,4.95976007523e-06
power_5,None,5.42247604482e-16,1.82593497407e-09
power_6,None,2.89563828343e-20,4.5209083829e-13
power_7,None,1.65000666351e-24,8.27825497639e-17
power_8,None,9.86081528409e-29,1.25871594306e-20
power_9,None,6.06589348254e-33,1.50535779641e-24


In [38]:
mod[1]['coefficients']

name,index,value,stderr
(intercept),None,519216.897383,2014830.5895
power_1,None,2.04470474182,11220.8415608
power_2,None,0.0011314362684,26.4066146726
power_3,None,2.93074277549e-07,0.034779163209
power_4,None,4.43540598453e-11,2.86275134176e-05
power_5,None,4.80849112204e-15,1.5529371824e-08
power_6,None,4.53091707826e-19,5.67268678875e-12
power_7,None,4.16042910575e-23,1.38721257101e-15
power_8,None,3.90094635128e-27,2.16335615706e-19
power_9,None,3.7773187602e-31,1.74349357506e-23


In [39]:
mod[2]['coefficients']

name,index,value,stderr
(intercept),None,522911.518048,1762190.05962
power_1,None,2.26890421877,8587.5632283
power_2,None,0.00125905041842,17.6448434043
power_3,None,2.77552918155e-07,0.0202285170891
power_4,None,3.2093309779e-11,1.44085645294e-05
power_5,None,2.87573572364e-15,6.68792040245e-09
power_6,None,2.50076112671e-19,2.04464086154e-12
power_7,None,2.24685265906e-23,3.99015927317e-16
power_8,None,2.09349983135e-27,4.41495522166e-20
power_9,None,2.00435383296e-31,2.89761327289e-24


In [35]:
mod[3]['coefficients']

name,index,value,stderr
(intercept),None,513667.087087,1373753.86668
power_1,None,1.91040938244,6702.74068754
power_2,None,0.00110058029175,6.94344669789
power_3,None,3.12753987879e-07,nan
power_4,None,5.50067886825e-11,nan
power_5,None,7.20467557825e-15,nan
power_6,None,8.24977249384e-19,nan
power_7,None,9.06503223498e-23,nan
power_8,None,9.95683160453e-27,nan
power_9,None,1.10838127982e-30,nan


These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [40]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [76]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    

Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [ ]:
train_valid_shuffled[0:10] # rows 0 to 9

Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [45]:
lst = [0,1939,3879,5818,7758,9698,11637,13577,15516,17456,None]
x = 'validation'
for y in range(1,11):
    string = x + str(y)
    globals()[string] = train_valid_shuffled[lst[y-1]:lst[y]] #still need to find out how to reduce calling global dict
                                                              #again and again frather just add to dictionary


To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [46]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [47]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 |  665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 |  645000.0 |   3.0    |    2.5    |
| 4139480190 | 2014-09-16 00:00:00+00:00 | 1153000.0 |   3.0    |    3.25   |
| 7237300290 | 2015-03-26 00:00:00+00:00 |  338000.0 |   5.0    |    2.5    |
+------------+---------------------------+-----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    2800.0   |   5900   |   1    |     0      |  0   |     3     |   8   |    1660    |
|    2490.0   |   59

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [57]:
train4 = graphlab.SFrame()
for x in range(1,11):
    if x == 4:
        continue
    string = 'validation' + str(x)
    train4 = train4.append(globals()[string])
    

In [58]:
len(train4.append(globals()['validation4']))

19396

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [59]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [70]:
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    count = 0
    fold_dict = {}
    for y in range(1,k+1):
        start = (n*(y-1))/k
        end = (n*y)/k-1
        string = 'fold' + str(y)
        fold_dict[string] = data[start:end]
    for x in fold_dict:
        validation = fold_dict[x]
        train = graphlab.SFrame()
        for e in fold_dict:
            if e == x:
                continue
            train = train.append(fold_dict[e])
        model = graphlab.linear_regression.create(train, target=output_name, l2_penalty=l2_penalty, features=features_list, validation_set=None,verbose=False) 
                                                      
        z = model.evaluate(validation)
        count = count + z['rmse']
    return (count/k)
        

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [75]:
lambdah = []
for x in np.logspace(1, 7, num=13):
    poly = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
    #print poly
    my_features = poly.column_names()
    #print my_features
    poly['price'] = train_valid_shuffled['price']
    lambdah.append(k_fold_cross_validation(10,x,poly,'price',my_features))
print lambdah

[364015.65331127343, 318508.8357052642, 274721.2278612014, 250322.49845733732, 249548.2869333804, 252406.8281025475, 265162.0216288048, 296927.2697167065, 341377.6542882941, 359675.7391688706, 364422.05787779216, 367351.99797094474, 368781.9653675688]


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

In [88]:
poly = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
    #print poly
my_features = poly.column_names()
    #print my_features
poly['price'] = train_valid_shuffled['price']

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [90]:
model = graphlab.linear_regression.create(poly, target='price', l2_penalty=1000, features=my_features, validation_set=None,verbose=False) 

In [94]:
(model.evaluate(test)['rmse']**2)*len(test)

252897427447157.44

In [ ]:
252897427447157.44